# First Iteration

In [ ]:
###################
# Setup and imports
###################

from reviews import reviews

from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

# select a comment to test with
review_text = reviews.critic_review_01

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents([review_text], metadatas=[{"source":"review_01"}])

embedding_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(docs, embedding_func)

retriever = db.as_retriever(search_kwargs={"k": 5})

from langchain_community.llms import Ollama

llm = Ollama(model='llama2:7b-chat-q4_K_M', temperature=0.2)        # set to a lower temperature for more consistent results

In [ ]:
###################
# First prompt
###################

from langchain.chains import RetrievalQAWithSourcesChain
from langchain_core.prompts import PromptTemplate

prompt_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question. 

{summaries}

Question: {question}

If you don't know the answer, output only "NA". Do NOT try to make up an answer. Do NOT output other text.'''

my_question_template = \
'''Extract the the following aspect of the game from the reviews. Output a paragraph with less than 200 words. The aspect is: '''

aspects = ['Gameplay', 'Sound', 'Graphics', 'Performance', 'Bug', 'Suggestion', 'Price', 'Overall']
aspects_response = {k: '' for k in aspects}

for aspect in aspects:
    my_question = my_question_template + f'{aspect}'
    print(my_question)



    chain =  RetrievalQAWithSourcesChain.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "prompt": PromptTemplate(
                template=prompt_template,
                input_variables=["summaries", "question"],
            )
        },
        return_source_documents=True,
    )

    response = chain.invoke(
        {
            'question': my_question
        }
    )

    print(response)
    print('\n\n')
    print(response['answer'])
    aspects_response[aspect] = response['answer']
    print('\n\n')
    print(response['source_documents'])

    print('\n\n\n')

Extract the the following aspect of the game from the reviews. Output a paragraph with less than 200 words. The aspect is: Gameplay
{'question': 'Extract the the following aspect of the game from the reviews. Output a paragraph with less than 200 words. The aspect is: Gameplay', 'answer': "\nAccording to John Tucker's review, the gameplay of Cyberpunk 2077 is excellent and allows players to choose their own approach to completing objectives. The reviewer mentions that they tend to play as a sneaky sniper type, but the game also accommodates those who prefer to be more aggressive. The review highlights the variety of options available to players, including the ability to infiltrate areas undetected or to use grenades and other weapons to take out enemies. Overall, the gameplay is praised for being engaging and allowing players to play the way they want to.\n\nNA", 'sources': '', 'source_documents': [Document(page_content='Cyberpunk 2077\nReview by John Tucker, January 3, 2021\n\nOver th

##### Gameplay
According to John Tucker's review, the gameplay of Cyberpunk 2077 is excellent and allows players to choose their own approach to completing objectives. The reviewer mentions that they tend to play as a sneaky sniper type, but the game also accommodates those who prefer to be more aggressive. The review highlights the variety of options available to players, including the ability to infiltrate areas undetected or to use grenades and other weapons to take out enemies. Overall, the gameplay is praised for being engaging and allowing players to play the way they want to.

##### Sound
The sound design in Cyberpunk 2077 is excellent, with a score of 90 out of 100. The reviewer praises the game's audio, stating that it is top-notch and one of the highlights of the game. They mention that the music and voice acting are particularly impressive, with only occasional glitches in the sound design. Overall, the sound design is a standout feature of the game.

##### Graphics
According to John Tucker's review, the graphics of Cyberpunk 2077 receive a score of 85 out of 100. The reviewer notes that while the game's graphics are impressive overall, there are some minor glitches and bugs that can affect gameplay. However, they emphasize that these issues do not detract significantly from the overall visual experience of the game.


##### Performance
According to John Tucker's review, the performance of Cyberpunk 2077 on console versions was "so buggy as to be unplayable," with digital and physical retailers expanding their refund policies to accommodate anyone who wanted to return the game. However, in his personal experience, he was able to play all the way to the end credits with only minor glitches on PC. Therefore, the answer is:

NA

##### Bug
Based on the reviews provided, the aspect of the game that can be extracted is: Bug. Here is a paragraph with less than 200 words summarizing the information:

The game has several bugs and glitches, including crashing, clipping NPCs, and unresponsive interactable objects. The reviewer experienced minor glitches on PC but had better luck overall compared to other reviewers who encountered more significant issues. Despite these problems, the reviewer notes that the game's presentation is top-notch overall, with excellent graphics, music, and voice acting.


##### Suggestion
According to the reviews, the game has some suggestions for players to consider. John Tucker mentions in his review that Cyberpunk 2077 is a game that can be frustrating to love due to its good design elements and great story, but also bad design choices and an unplayable console version. However, he hopes to find a balance between factoring in these elements and leaving them out unless they impact his particular experience. In the end, he suggests that if you can take the game's flaws and remember to do better in your own life, there is an amazing game here that you can be glad you experienced.

NA
##### Price
According to the reviews, the price of Cyberpunk 2077 is NA. None of the reviewers mention the game's price, and it does not appear to be a significant factor in their overall assessment of the game.

##### Overall
According to the reviews, the overall score of Cyberpunk 2077 is 70 out of 100. The reviewers had mixed opinions about the game's overall quality, with some praising its engaging story and immersive world, while others criticized its buggy performance and poorly implemented features. NA



---

Shorten the sentences and output as a JSON object

In [ ]:
####################
# Second Prompt
####################

from langchain_core.prompts import ChatPromptTemplate

system_template = \
'''You are reading reviews of a game to understand the characteristics of the game. Use the following pieces of context to answer user's question.
'''

summary_template = \
'''Extract the following aspects of the game from the reviews, and write a short 20 words description for each aspect. The aspects are: [Gameplay, Graphics, Sound, Performance, Bug, Suggestion, Price, Overall]. Output a JSON with each of the aspects as key, and the information as the value. Only output the JSON. Do NOT output other text.

The context is wrapped by three consecutive apostrophes. The context is as follows:
\'\'\'
{context}
\'\'\'
'''

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", summary_template),
])

chain = chat_prompt | llm
response = chain.invoke({"context":str(aspects_response)})

print(response)

{
"Gameplay": "Excellent with variety of options for players to choose their own approach.",
"Graphics": "Impressive overall but with minor glitches and bugs.",
"Sound": "Top-notch audio, including music and voice acting.",
"Performance": "Buggy on console versions, but playable on PC with minor glitches.",
"Bug": "Several bugs and glitches, including crashing, clipping NPCs, and unresponsive interactable objects.",
"Suggestion": "A game that can be frustrating to love due to good design elements and great story, but bad design choices and an unplayable console version. Hope to find a balance between factoring in these elements and leaving them out unless they impact the particular experience.",
"Price": "NA",
"Overall": "70 out of 100 with mixed opinions on overall quality."
}
